<a href = "https://www.pieriantraining.com"><img src="../PT Centered Purple.png"> </a>

<em style="text-align:center">Copyrighted by Pierian Training</em>

# Budget Management for Azure with Python

## Azure Actions Covered

* List and get budgets
* Create and delete budgets
* List usage details
* List charges

In this lecture, we'll learn how to use the Azure budget management library with Python.

To begin, we'll need to import our usual libraries as well as any useful environment variables (e.g. AZURE_SUBSCRIPTION_ID). We'll add some new imports as well.

In [1]:
import datetime

from azure.identity import AzureCliCredential
# New imports for budget management
from azure.mgmt.consumption import ConsumptionManagementClient
from azure.mgmt.consumption.models import Budget, BudgetTimePeriod, Notification

from settings import AZURE_SUBSCRIPTION_ID, DEFAULT_RESOURCE_GROUP, DEFAULT_LOCATION

Let's instantiate our credential and then use it to instantiate the `ConsumptionManagementClient`.

In [2]:
credential = AzureCliCredential()
cm_client = ConsumptionManagementClient(credential, AZURE_SUBSCRIPTION_ID)

We can use the `ConsumptionManagementClient` to list any budgets that we've already created in our Azure portal (or via the Azure CLI). The returned objects are `Budget` objects, which have some useful parameters that we'll print out. For a full list of parameters, see the [Budget class](https://learn.microsoft.com/en-us/python/api/azure-mgmt-consumption/azure.mgmt.consumption.models.budget?view=azure-python).

In [3]:
print('Budget Name - Category - Time Grain - Amount')
print('--------------------------------------------')
for budget in cm_client.budgets.list(f'/subscriptions/{AZURE_SUBSCRIPTION_ID}/'):
    print(f'{budget.name} - {budget.category} - {budget.time_grain} - {budget.amount}')

Budget Name - Category - Time Grain - Amount
--------------------------------------------
bens-budget - Cost - Monthly - 15.0
bens-budget-2 - Cost - Quarterly - 25.0


Once we know what some of our budget names are, we can use the `get()` method to retrieve the `Budget` object so that we can interact with some of its properties and parameters.

In [8]:
my_budget = cm_client.budgets.get(
    scope=f'/subscriptions/{AZURE_SUBSCRIPTION_ID}', 
    budget_name='bens-budget'
)

For example, we can look at our current spend under our budget.

In [10]:
my_budget.current_spend.as_dict()

{'amount': 0.0, 'unit': 'USD'}

We can also get the budget amount/limit.

In [11]:
my_budget.amount

15.0

We can also look at the notifications that have been set up for a particular budget.

In [12]:
my_budget.notifications

{'actual_GreaterThan_50_Percent': <azure.mgmt.consumption.models._models_py3.Notification at 0x7f9674daf700>,
 'forecasted_GreaterThan_75_Percent': <azure.mgmt.consumption.models._models_py3.Notification at 0x7f9674daf970>}

Now let's create a new budget with the `create_or_update()` method. We'll need to fill in the following parameters:

* `scope` - Scope of the budget. For example: subscription, resource group, billing account, etc.
* `budget_name` - Name for the new budget
* `parameters` - Budget parameters, which can be a `Budget` object. These include:
    * `category` - Category of budget; right now, this can only be "cost"
    * `amount` - Total amount of cost to track (e.g. in USD)
    * `time_grain` - Time granularity for the budget. For example: "Monthly", "Quarterly", etc.
    * `time_period` - Start and end date of the budget 
    * `notifications` - Dictionary of notifications for the budget. Dict is of type `{str: Notification}`
    
Again, for the full list of `Budget` parameters, see the [Budget class](https://learn.microsoft.com/en-us/python/api/azure-mgmt-consumption/azure.mgmt.consumption.models.budget?view=azure-python).

For a full list of `Notification` parameters, see the [Notification class](https://learn.microsoft.com/en-us/python/api/azure-mgmt-consumption/azure.mgmt.consumption.models.notification?view=azure-python).

In [4]:
new_budget = cm_client.budgets.create_or_update(
    scope=f'/subscriptions/{AZURE_SUBSCRIPTION_ID}/',
    budget_name='bens-budget-2',
    parameters=Budget(
        category="cost",
        amount=25.0,
        time_grain="Quarterly",
        time_period=BudgetTimePeriod(
            start_date=datetime.datetime(2023, 6, 1),
            end_date=datetime.datetime(2023, 12, 31)
        ),
        notifications={
            'my_notification': Notification(
                enabled=True,
                operator='GreaterThan',
                threshold=50.0,
                contact_emails=['first.last@mail.com'],
                threshold_type='Actual'
            )
        }
    )
)

Datetime with no tzinfo will be considered UTC.
Datetime with no tzinfo will be considered UTC.


Let's take a look at the returned parameters, which will mirror what we provided to the `Budget` object.

In [ ]:
new_budget.as_dict()

We can also delete budgets via Python with the `delete()` method. Let's delete the budget we just created.

In [20]:
cm_client.budgets.delete(
    scope=f'/subscriptions/{AZURE_SUBSCRIPTION_ID}',
    budget_name='bens-budget-2'
)

The `ConsumptionManagementClient` also provides access to our usage details. We can use the `list()` method to show all of our usage details. There are two types of usage details:

* Modern - For full list of parameters, see the [ModernUsageDetail class](https://learn.microsoft.com/en-us/python/api/azure-mgmt-consumption/azure.mgmt.consumption.models.modernusagedetail?view=azure-python)
* Legacy - For full list of parameters, see the [LegacyUsageDetail class](https://learn.microsoft.com/en-us/python/api/azure-mgmt-consumption/azure.mgmt.consumption.models.legacyusagedetail?view=azure-python).

Some examples of the data provided with usage details:

* `name` - Resource name
* `product` - Name of the Azure product that has accrued charges
* `service_family` - Service family for the product
* `quantity` - Quantity of product/service consumed
* `cost_in_billing_currency` - Cost of resource in billed currency

In [6]:
for detail in cm_client.usage_details.list(f'/subscriptions/{AZURE_SUBSCRIPTION_ID}/'):
    print(f'{detail.product}')
    print(f'{detail.meter_category} - {detail.service_family}')
    print(f'{detail.cost_in_billing_currency}')

Bandwidth Inter-Region - Intra Continent - North America
Bandwidth - Networking
2.4e-06
Tables - LRS
Storage - Storage
5.4e-06
Blob Storage - Hot LRS - US East
Storage - Storage
2.4e-06
Tables - LRS
Storage - Storage
5.256e-06
General Block Blob v2 Hierarchical Namespace - Hot RA-GRS - US East
Storage - Storage
7.08e-06
Tables - RA-GRS
Storage - Storage
5.7e-06
Storage - Bandwidth - Geo-Replication - US East
Storage - Storage
0.0
Tables - GRS
Storage - Storage
1.0656e-05
Blob Storage - Hot LRS - US East
Storage - Storage
2.4e-06
Tables - LRS
Storage - Storage
5.256e-06
General Block Blob v2 Hierarchical Namespace - Hot RA-GRS - US East
Storage - Storage
7.08e-06
Storage - Bandwidth - Geo-Replication - US East
Storage - Storage
0.0
Tables - GRS
Storage - Storage
1.044e-05
Tables - RA-GRS
Storage - Storage
9.6e-06
Azure App Service Free Plan - Linux - F1
Azure App Service - Compute
0.0
General Block Blob v2 Hierarchical Namespace - Hot LRS - US East
Storage - Storage
7e-06
Bandwidth Inte

We can also list charges, but these must be associated with a billing account ID.

In [7]:
BILLING_ACCOUNT_ID = '<insert billing account id>'
charges = cm_client.charges.list(f'/providers/Microsoft.Billing/billingAccounts/{BILLING_ACCOUNT_ID}/')

Just like for usage details, our charge summaries have **Legacy** and **Modern** versions.

* Legacy - For the full list of parameters, see the [LegacyChargeSummary class](https://learn.microsoft.com/en-us/python/api/azure-mgmt-consumption/azure.mgmt.consumption.models.legacychargesummary?view=azure-python).
* Modern - For the full list of parameters, see the [ModernChargeSummary class](https://learn.microsoft.com/en-us/python/api/azure-mgmt-consumption/azure.mgmt.consumption.models.modernchargesummary?view=azure-python).

We'll print out some of our charges from the returned list.

In [18]:
charges.value

Let's take a look at the `azure_charges`.

In [19]:
charges.value[1].azure_charges.as_dict()

{'currency': 'USD', 'value': 0.342}